In [13]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import validation
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
!pip install eto
from eto import ETo,datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 7.3 MB/s 
  Created wheel for eto: filename=ETo-1.1.0-py2.py3-none-any.whl size=113469 sha256=db4638cfe41ddc14cc7fe882567d70b9a37e546fd2371360f19ac5ab660123eb
  Stored in directory: /root/.cache/pip/wheels/27/3c/85/4cfc3f25ed4adbc404fc676105e52f6840d8c22cb2f28070f0
Successfully built eto


In [14]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
final_data= drive.CreateFile({'id': '1MmTTODOQVsNw9F5ljMMmHcu8xElnkpxO'});
final_data.GetContentFile('final_data.csv');

In [15]:
Final_data =pd.read_csv('final_data.csv', parse_dates=True, infer_datetime_format=True, index_col='date')
ET1=ETo()
ET1.param_est(Final_data,freq='D',z_msl=257,lat=30.3398,z_u=2.0)
ET1.ts_param.head()
Features=ET1.ts_param
Features=Features[['T_max','T_min','U_z','RH_min','RH_max','R_n','n_sun']]
Features['RH']=(Features['RH_min']+Features['RH_max'])/2
Features=Features.drop(['RH_min','RH_max'],axis=1)
print(Features.isnull().sum())
ETO=ET1.eto_fao()
Output=pd.DataFrame(ETO)
Output=Output.interpolate(method='linear',limit=64)
print(Output.isnull().sum())

T_max    0
T_min    0
U_z      0
R_n      0
n_sun    0
RH       0
dtype: int64
ETo_FAO_mm    0
dtype: int64


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Random Forest Model

In [16]:
xtrain,xtest,ytrain,ytest=train_test_split(Features,Output,test_size=0.3)
RandomForest=RandomForestRegressor()
model=RandomForest.fit(xtrain,ytrain)
result=model.score(xtest,ytest)
print(f"Accuracy: {result*100}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy: 98.44673532940872


In [17]:
# Calculating r2 score
ypred=model.predict(xtest)
print(r2_score(ytest,ypred))
#calculating RMLSE Root Mean Squared Logarithmic Error used when numbers are too large..else RMSE or standard deviation and RMLSE are nearly same
print(mean_squared_log_error(ytest, ypred))
# calculating NSE Nash–Sutcliffe model efficiency coefficient use to access skills of models
ypred=pd.DataFrame(ypred)
ytest=pd.DataFrame(ytest)
def nse(predictions, targets):
  num=0
  denom=0
  for i in range(len(ypred)):
    num=num+(ypred.iloc[i][0]-ytest.iloc[i][0])**2
    denom=denom+(ypred.iloc[i][0]-np.mean(ypred))**2
  return(1-num/denom)
print(nse(ypred,ytest))
#calculating RAE Relative Absolute Error
import math
ypred=pd.DataFrame(ypred)
ytest=pd.DataFrame(ytest)
def rae(predictions, targets):
  num=0
  denom=0
  for i in range(len(ypred)):
    num=num+(ypred.iloc[i][0]-ytest.iloc[i][0])**2
    denom=denom+(ytest.iloc[i][0])**2
  math.sqrt(num)
  math.sqrt(denom)
  return(1-num/denom)
print(rae(ypred,ytest))

0.9844673532940873
0.003755740408870891
0    0.983886
dtype: float64
0.996394949889505


Deep Learning Model

In [ ]:
model=Sequential()
xtrain,xtest,ytrain,ytest=train_test_split(Features,Output,test_size=0.3)
xtrain=xtrain.to_numpy()
xtest=xtest.to_numpy()
ytrain=ytrain.to_numpy()
ytest=ytest.to_numpy()
model.add(Dense(250,activation='relu',input_dim=6))
model.add(Dense(500,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1000,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(250,activation='relu'))
model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))
model.fit(xtrain,ytrain,epochs=100)

In [ ]:
# Calculating r2 score
ypred=model.predict(xtest)
print(r2_score(ytest,ypred))
#calculating RMLSE Root Mean Squared Logarithmic Error used when numbers are too large..else RMSE or standard deviation and RMLSE are nearly same
print(mean_squared_log_error(ytest, ypred))
# calculating NSE Nash–Sutcliffe model efficiency coefficient use to access skills of models
ypred=pd.DataFrame(ypred)
ytest=pd.DataFrame(ytest)
def nse(predictions, targets):
  num=0
  denom=0
  for i in range(len(ypred)):
    num=num+(ypred.iloc[i][0]-ytest.iloc[i][0])**2
    denom=denom+(ypred.iloc[i][0]-np.mean(ypred))**2
  return(1-num/denom)
print(nse(ypred,ytest))
#calculating RAE Relative Absolute Error
import math
ypred=pd.DataFrame(ypred)
ytest=pd.DataFrame(ytest)
def rae(predictions, targets):
  num=0
  denom=0
  for i in range(len(ypred)):
    num=num+(ypred.iloc[i][0]-ytest.iloc[i][0])**2
    denom=denom+(ytest.iloc[i][0])**2
  math.sqrt(num)
  math.sqrt(denom)
  return(1-num/denom)
print(rae(ypred,ytest))

Gradient Boosting Machine

In [ ]:
GB=GradientBoostingRegressor()
paramas={'n_estimators':range(1,100)}
grid=GridSearchCV( estimator=GB,param_grid=paramas,scoring='neg_mean_squared_error') 
grid.fit(xtest,ytest)
print(grid.best_estimator_)
model=grid.best_estimator_
ypred=model.predict(xtest)

In [19]:
# Calculating r2 score
ypred=model.predict(xtest)
print(r2_score(ytest,ypred))
#calculating RMLSE Root Mean Squared Logarithmic Error used when numbers are too large..else RMSE or standard deviation and RMLSE are nearly same
print(mean_squared_log_error(ytest, ypred))
# calculating NSE Nash–Sutcliffe model efficiency coefficient use to access skills of models
ypred=pd.DataFrame(ypred)
ytest=pd.DataFrame(ytest)
def nse(predictions, targets):
  num=0
  denom=0
  for i in range(len(ypred)):
    num=num+(ypred.iloc[i][0]-ytest.iloc[i][0])**2
    denom=denom+(ypred.iloc[i][0]-np.mean(ypred))**2
  return(1-num/denom)
print(nse(ypred,ytest))
#calculating RAE Relative Absolute Error
import math
ypred=pd.DataFrame(ypred)
ytest=pd.DataFrame(ytest)
def rae(predictions, targets):
  num=0
  denom=0
  for i in range(len(ypred)):
    num=num+(ypred.iloc[i][0]-ytest.iloc[i][0])**2
    denom=denom+(ytest.iloc[i][0])**2
  math.sqrt(num)
  math.sqrt(denom)
  return(1-num/denom)
print(rae(ypred,ytest))

0.9938056647839153
0.0017054776430627035
0    0.99373
dtype: float64
0.9985623255792789


GLM Model


In [20]:
GLM_model=ElasticNet(alpha=0.4,l1_ratio=0.7)
GLM_model.fit(xtrain,ytrain)
ypred=GLM_model.predict(xtest)

In [21]:
# Calculating r2 score
ypred=model.predict(xtest)
print(r2_score(ytest,ypred))
#calculating RMLSE Root Mean Squared Logarithmic Error used when numbers are too large..else RMSE or standard deviation and RMLSE are nearly same
print(mean_squared_log_error(ytest, ypred))
# calculating NSE Nash–Sutcliffe model efficiency coefficient use to access skills of models
ypred=pd.DataFrame(ypred)
ytest=pd.DataFrame(ytest)
def nse(predictions, targets):
  num=0
  denom=0
  for i in range(len(ypred)):
    num=num+(ypred.iloc[i][0]-ytest.iloc[i][0])**2
    denom=denom+(ypred.iloc[i][0]-np.mean(ypred))**2
  return(1-num/denom)
print(nse(ypred,ytest))
#calculating RAE Relative Absolute Error
import math
ypred=pd.DataFrame(ypred)
ytest=pd.DataFrame(ytest)
def rae(predictions, targets):
  num=0
  denom=0
  for i in range(len(ypred)):
    num=num+(ypred.iloc[i][0]-ytest.iloc[i][0])**2
    denom=denom+(ytest.iloc[i][0])**2
  math.sqrt(num)
  math.sqrt(denom)
  return(1-num/denom)
print(rae(ypred,ytest))

0.9938056647839153
0.0017054776430627035
0    0.99373
dtype: float64
0.9985623255792789
